# **Modélisation 2**

À partir de l'article Medium référencé dans le README, nous allons essayer de créer un algorithme de recommendation à partir de notre base. Au vu du contenu de notre base, nous n'allons pas vraimet pouvoir implementer une partie en fonction de ce que les utilisateurs ayant aimé un film auraient aussi aimé, et plus nous concentrer sur les caractéristiques des films.

In [34]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [14]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

def fonction_charge(path) :
    df = pd.read_csv(path)
    df_f = df.drop_duplicates()
    df_f['release_date'] = pd.to_datetime(df_f['release_date'])
    # De la même façon on convertit la colonne 'budget' qui est composée de chaînes de caractères et non de valeurs numériques
    df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
    df_f['genres_list'] = df_f['genres_list'].apply(ast.literal_eval)
    df_f['countries_prod'] = df_f['countries_prod'].apply(ast.literal_eval)
    df_f['languages_list'] = df_f['languages_list'].apply(ast.literal_eval)
    df_f['prod_companies'] = df_f['prod_companies'].apply(ast.literal_eval)
    df_f['directors'] = df_f['directors'].apply(safe_literal_eval)
    return df_f 

In [20]:
df = fonction_charge('../cleaning_data/Final_database.csv')

/tmp/ipykernel_58073/1525667594.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['release_date'] = pd.to_datetime(df_f['release_date'])
/tmp/ipykernel_58073/1525667594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
/tmp/ipykernel_58073/1525667594.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [29]:
movies_df = df
movies_df['genres_ex'] = movies_df['genres_list'].apply(lambda x: ', '.join(map(str, x)))
movies_df['direct_ex'] = movies_df['directors'].apply(lambda x: ', '.join(map(str, x)))
movies_df
# Create a new column with combined information
movies_df['combined_info'] = (
    movies_df['overview'] + ' ' + movies_df['genres_ex'] + ' ' +  movies_df['direct_ex'] + ' ' +  movies_df['collection_name'].fillna('')
)
movies_df['combined_info']


0        An ex-special forces operative takes a job to ...
1        When oil is discovered in 1920s Oklahoma under...
2        When Branch's brother, Floyd, is kidnapped for...
3        Jaded 74-year-old lizard Leo has been stuck in...
4        Willy Wonka – chock-full of ideas and determin...
                               ...                        
12012    Ten years ago, over forty people were brutally...
12013    An ex-soldier, a teen and a cop collide in New...
12014    In London, after investigating crack addicted ...
12015    Animated character Fat Albert emerges from his...
12016    A junkie must face his true self to kick his d...
Name: combined_info, Length: 9180, dtype: object

In [33]:
movies_df['combined_info'].fillna('', inplace=True)

# Initialize the Object and remove stopwords
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['combined_info'])


In [35]:
# compute the similarity matirx
cos_mat = linear_kernel(tfidf_matrix, tfidf_matrix)

In [36]:
diag = 0
for i in range(len(cos_mat)):
    diag+= cos_mat[i][i]
    
print(diag)

9158.0


In [37]:
def get_recommendations(movie, n):
    
    # get index from dataframe
    index = movies_df[movies_df['title']== movie].index[0]
    
    # sort top n similar movies     
    similar_movies = sorted(list(enumerate(cos_mat[index])), reverse=True, key=lambda x: x[1]) 
    
    # extract names from dataframe and return movie names
    recomm = []
    for i in similar_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm


In [45]:
get_recommendations("Leo", 5)

['The Week Of',
 'Cat Person',
 'Notes on a Scandal',
 'Life in a Year',
 'Home Alone 2: Lost in New York']

In [41]:
# Cosine similarity not defined !! Donc ça ne fonctionne pas !! À reprendre

def get_keywords_recommendations(keywords, n):
    
    keywords = keywords.split()
    keywords = " ".join(keywords)
    
    # transform the string to vector representation
    key_tfidf = tfidf.transform([keywords]) 
    
    # compute cosine similarity    
    result = cosine_similarity(key_tfidf, cos_mat)
    
    # sort top n similar movies   
    similar_key_movies = sorted(list(enumerate(result[0])), reverse=True, key=lambda x: x[1])
    
    # extract names from dataframe and return movie names
    recomm = []
    for i in similar_key_movies[1:n+1]:
        recomm.append(df.iloc[i[0]].title)
        
    return recomm

In [42]:
get_keywords_recommendations("Christopher Nolan", 4)

NameError: name 'cosine_similarity' is not defined